In [1]:
!gdown 1-36YC-ejOi8nIB8UdwSInyRhh96T2ThX
!unzip /content/kaggle_mmseg.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/deform_roi_pool_cuda_kernel.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/knn_cuda_kernel.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/roi_pool_cuda_kernel.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/furthest_point_sample_cuda_kernel.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/nms_cuda_kernel.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/ms_deform_attn_cuda_kernel.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/min_area_polygons_cuda.cuh  
  inflating: openmmlab-essential-repositories/openmmlab-repos/mmcv/mmcv/ops/csrc/common/cuda/iou3d_cuda_ke

In [18]:
!gdown 1-KPxOC-9yvG9IYf9C3zsRMS8aZ2xeSl5
!unzip /content/conv_origin_512_r2_binary_dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1-5vOf41Zq-o1Pg8dKzbJ3pI02dIWR1sm
To: /content/256_4_stride64.zip
100% 3.57G/3.57G [00:16<00:00, 217MB/s]


In [5]:
!pip install /content/openmmlab-essential-repositories/openmmlab-repos/src/torch-1.10.0+cu111-cp37-cp37m-linux_x86_64.whl
!pip install /content/openmmlab-essential-repositories/openmmlab-repos/src/torchvision-0.11.0+cu111-cp37-cp37m-linux_x86_64.whl
!pip install /content/openmmlab-essential-repositories/openmmlab-repos/src/mmcv_full-1.5.3-cp37-cp37m-manylinux1_x86_64.whl
!pip install /content/openmmlab-essential-repositories/openmmlab-repos/src/mmcls-0.23.1-py2.py3-none-any.whl
!pip install /content/mmdetection/addict-2.4.0-py3-none-any.whl
!pip install /content/mmdetection/yapf-0.31.0-py2.py3-none-any.whl
!pip install /content/mmdetection/terminaltables-3.1.0-py3-none-any.whl
!pip install /content/mmdetection/einops-0.4.1-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./openmmlab-essential-repositories/openmmlab-repos/src/torch-1.10.0+cu111-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.10.0+cu111 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.10.0+cu111 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.10.0+cu111 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./openmmlab-essential-repositor

In [6]:
!mkdir -p ./mmseg_data/images
!mkdir -p ./mmseg_data/labels
!mkdir -p ./mmseg_data/splits

In [7]:
!cp -r /content/train/* /content/mmseg_data/images/
!cp -r /content/masks/* /content/mmseg_data/labels/

In [8]:
from glob import glob
import numpy as np
import cv2
import os
from sklearn.model_selection import StratifiedKFold
Fold = 5
all_mask_files = glob("./mmseg_data/labels/*")
masks = []
num_wo_mask = np.zeros(Fold)
num_w_mask = np.zeros(Fold)
for i in range(len(all_mask_files)):
    mask = cv2.imread(all_mask_files[i])
    if mask.sum()==0:
        masks.append(0)
    else:
        masks.append(1)
split = list(StratifiedKFold(n_splits=Fold, shuffle=True, random_state=2022).split(all_mask_files, masks))
for fold, (train_idx, valid_idx) in enumerate(split):
    for i in valid_idx:
        if masks[i]==0:
            num_wo_mask[fold]+=1
        else:
            num_w_mask[fold]+=1
    with open(f"./mmseg_data/splits/fold_{fold}.txt", "w") as f:
        for idx in train_idx:
            f.write(os.path.basename(all_mask_files[idx])[:-4] + "\n")
    with open(f"./mmseg_data/splits/valid_{fold}.txt", "w") as f:
        for idx in valid_idx:
            f.write(os.path.basename(all_mask_files[idx])[:-4] + "\n")
print(num_wo_mask)
print(num_w_mask)

[202. 202. 202. 202. 202.]
[377. 377. 377. 377. 377.]


In [9]:
!mkdir ./working

In [10]:
!cp -r ./mmsegm/mmsegmentation-master ./working/ && cd ./working/mmsegmentation-master && pip install -e . && cd ..

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/working/mmsegmentation-master
  Running setup.py develop for mmsegmentation


In [12]:
%%bash

cat <<EOT >> /content/working/config.py

norm_cfg = dict(type='BN', requires_grad=True)
checkpoint = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segformer/mit_b5_20220624-658746d9.pth'
model = dict(
    type='EncoderDecoder',
    pretrained=checkpoint,
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1),
    decode_head=dict(
        type='SegformerHead',
        in_channels=[64, 128, 320, 512],
        in_index=[0, 1, 2, 3],
        channels=256,
        dropout_ratio=0.1,
        num_classes=2,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    # model training and testing settings
    train_cfg=dict(),
    test_cfg=dict(mode='slide', crop_size=(512, 512), stride=(512, 512)))

# dataset settings
dataset_type = 'CustomDataset'
data_root = './mmseg_data/'
classes = ['BG', 'FTU']
palette = [[0,0,0], [255,0,0]]
img_norm_cfg = dict(mean=[196.869, 190.186, 194.802], std=[63.010, 66.765, 65.745], to_rgb=True)
size = 256

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(size, size), keep_ratio=True),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=(size, size), pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(size, size),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size=(size, size), pad_val=0, seg_pad_val=255),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=8,
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images',
        ann_dir='labels',
        img_suffix=".png",
        seg_map_suffix='.png',
        split="splits/fold_0.txt",
        classes=classes,
        palette=palette,
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images',
        ann_dir='labels',
        img_suffix=".png",
        seg_map_suffix='.png',
        split="splits/valid_0.txt",
        classes=classes,
        palette=palette,
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        data_root=data_root,
        test_mode=True,
        img_dir='test/images',
        ann_dir='test/labels',
        img_suffix=".png",
        seg_map_suffix='.png',
        classes=classes,
        palette=palette,
        pipeline=test_pipeline))

# yapf:disable
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook', by_epoch=False),
    ])
# yapf:enable
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
cudnn_benchmark = True

total_iters = 50000
# optimizer
optimizer = dict(
    type='AdamW',
    lr=0.00006,
    betas=(0.9, 0.999),
    weight_decay=0.01,
    paramwise_cfg=dict(
        custom_keys={
            'pos_block': dict(decay_mult=0.),
            'norm': dict(decay_mult=0.),
            'head': dict(lr_mult=10.)
        }))
# optimizer_config = dict(type='Fp16OptimizerHook', loss_scale='dynamic') # base code
optimizer_config = dict() # change code
# learning policy
lr_config = dict(
      _delete_=True,
      policy='poly',
      warmup='linear',
      warmup_iters=1500,
      warmup_ratio=1e-6,
      power=1.0,
      min_lr=0.0,
      by_epoch=False)
# runtime settings
find_unused_parameters = True
runner = dict(type = 'IterBasedRunner', max_iters = total_iters)
checkpoint_config = dict(by_epoch=False, interval=-1, save_optimizer=False)
# evaluation = dict(by_epoch=False, interval=500, metric='mDice', pre_eval=True) # base code
evaluation = dict(by_epoch=False, interval=50, metric='mDice', save_best='mDice') # change code
# fp16 = dict()
work_dir = './baseline'

EOT

In [17]:
!python ./working/mmsegmentation-master/tools/train.py ./working/config.py

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
|  aAcc | mDice |  mAcc |
+-------+-------+-------+
| 97.33 | 92.28 | 91.69 |
+-------+-------+-------+
2022-09-17 16:51:48,433 - mmseg - INFO - Iter(val) [579]	aAcc: 0.9733, mDice: 0.9228, mAcc: 0.9169, Dice.BG: 0.9852, Dice.FTU: 0.8603, Acc.BG: 0.9869, Acc.FTU: 0.8470
2022-09-17 16:52:18,083 - mmseg - INFO - Iter [35450/50000]	lr: 1.746e-05, eta: 5:42:49, time: 1.392, data_time: 0.805, memory: 5231, decode.loss_ce: 0.0058, decode.acc_seg: 99.7500, loss: 0.0058
[>>] 579/579, 16.8 task/s, elapsed: 35s, ETA:     0s2022-09-17 16:52:57,855 - mmseg - INFO - per class results:
2022-09-17 16:52:57,856 - mmseg - INFO - 
+-------+-------+-------+
| Class |  Dice |  Acc  |
+-------+-------+-------+
|   BG  | 98.52 | 98.68 |
|  FTU  | 86.01 | 84.75 |
+-------+-------+-------+
2022-09-17 16:52:57,856 - mmseg - INFO - Summary:
2022-09-17 16:52:57,856 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc | mDice |  mAcc |
+-------+-------+-------+
| 97.32 | 92.27 |

In [19]:
pth_files = 'mmseg_segformer_test'

!mkdir {pth_files}

In [20]:
!zip /content/drive/MyDrive/data/kaggle/main_test/mmseg/{pth_files}.zip -r ./baseline

  adding: baseline/ (stored 0%)
  adding: baseline/iter_50000.pth (deflated 7%)
  adding: baseline/best_mDice_iter_21150.pth (deflated 7%)
  adding: baseline/20220917_025112.log (deflated 96%)
  adding: baseline/20220917_024821.log (deflated 96%)
  adding: baseline/20220917_025010.log (deflated 96%)
  adding: baseline/20220917_025308.log.json (deflated 69%)
  adding: baseline/latest.pth (deflated 7%)
  adding: baseline/config.py (deflated 76%)
  adding: baseline/20220917_025655.log.json (deflated 89%)
  adding: baseline/20220917_025655.log (deflated 91%)
  adding: baseline/20220917_025308.log (deflated 96%)


In [63]:
!mv ./baseline/iter_50000.pth ./{pth_files}

In [34]:
file_name = 'mmseg_test1'

In [39]:
!mkdir {file_name}
!mv ./baseline/config.py ./{file_name}
!mv ./baseline/best_mDice_iter_5000.pth ./{file_name}

mkdir: cannot create directory ‘mmseg_test1’: File exists


In [64]:
!zip /content/drive/MyDrive/data/kaggle/main_test/mmseg/{pth_files}.zip -r ./{pth_files}

  adding: pth_files2/ (stored 0%)
  adding: pth_files2/iter_50000.pth (deflated 7%)


# Inference